In [1]:
!pip -q install PyGeodesy
!pip -q install numpy
!pip -q install polliwog
!pip -q install folium

In [2]:
from math import pi, sqrt, radians, degrees

import numpy as np

from pygeodesy.datum import Ellipsoid, Ellipsoids
from pygeodesy.vector3d import Vector3Tuple

WGS84 = Ellipsoids.WGS84
KM = 1000

from polliwog.transform.composite import CompositeTransform
from polliwog.transform.rotation import rotation_from_up_and_look, euler

import folium

class SpaceObject:
    def __str__(self):
        return """
            Lat (deg): {}
            Lon (deg): {}
            Alt (m):   {}
            """.format(self.lat_d, self.lon_d, self.alt)

class Beam(SpaceObject):
    def __init__(self):
        self.alt = 0.0
        self.width_d = 0.5

def dist(a, b):
    return sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2 + (b[2] - a[2])**2)

def normalized(array):
    return array/np.linalg.norm(array)

def los_to_earth(position, pointing, ellipsoid):
    """
        Adapted from Stephen Hartzell
    """
    pointing_norm = normalized(pointing)

    a = ellipsoid.a
    b = ellipsoid.b
    c = ellipsoid.c
    x = position[0]
    y = position[1]
    z = position[2]
    u = pointing_norm[0]
    v = pointing_norm[1]
    w = pointing_norm[2]

    value = -a**2*b**2*w*z - a**2*c**2*v*y - b**2*c**2*u*x
    radical = a**2*b**2*w**2 + a**2*c**2*v**2 - a**2*v**2*z**2 + \
     2*a**2*v*w*y*z - a**2*w**2*y**2 + b**2*c**2*u**2 - b**2*u**2*z**2 + \
     2*b**2*u*w*x*z - b**2*w**2*x**2 - c**2*u**2*y**2 + 2*c**2*u*v*x*y - c**2*v**2*x**2
    magnitude = a**2*b**2*w**2 + a**2*c**2*v**2 + b**2*c**2*u**2

    if radical < 0:
        return (None, False)

    d = (value - a*b*c*np.sqrt(radical)) / magnitude

    if d < 0:
        return (None, False)

    result = np.array([
        x + d * u,
        y + d * v,
        z + d * w,
    ])
    
    return (result, True)


def ecef_forward(geopoint):
    (x, y, z, _, _, _, _, _, _) = WGS84.ecef().forward(geopoint.lat_d, geopoint.lon_d, geopoint.alt)
    return np.array([x, y, z])

def ecef_reverse(ecef_point):
    (_, _, _, lat, lon, height, _, _, _) = WGS84.ecef().reverse(ecef_point[0], ecef_point[1], ecef_point[2])
    o = SpaceObject()
    o.lat_d = lat
    o.lon_d = lon
    o.alt = height
    return o

def beam_shifter(sat, beam):
    center = np.array([0.0, 0.0, 0.0])

    r = WGS84.Rgeocentric(beam.lat_d)
    r0 = WGS84.Rgeocentric(lat=0.0)

    sat_ecef = ecef_forward(sat)
    beam_ecef = ecef_forward(beam)

    z_up = np.array([0.0, 0.0, 1.0])
    towards = normalized(beam_ecef - sat_ecef)

    to_sat_frame = CompositeTransform()
    to_sat_frame.translate(-sat_ecef)
    to_sat_frame.append_transform3(
        rotation_from_up_and_look(
            z_up, 
            towards
        )
    )

    beam_sat_frame = to_sat_frame(beam_ecef)

    def shifted_beam(sx, sy):
        shift = CompositeTransform()
        shift.append_transform3(euler([-sx, -sy, 0.0]))

        beam_shift_sat_frame = shift(beam_sat_frame)
        shifted_beam_ecef = to_sat_frame(beam_shift_sat_frame, reverse=True)
        shifted_beam_isect_ecef, ok = los_to_earth(sat_ecef, shifted_beam_ecef - sat_ecef, WGS84)

        if not ok:
            return (None, ok)

        return (ecef_reverse(shifted_beam_isect_ecef), True)

    return shifted_beam

def footprint(sat, beam):
    shifter = beam_shifter(sat, beam) 

    rotate_rad = np.linspace(0, 2*pi, 150)
    shift_x_rad = np.arctan(np.sin(rotate_rad)*np.tan(radians(beam.width_d)))
    shift_y_rad = np.arctan(np.cos(rotate_rad)*np.tan(radians(beam.width_d)))

    result = [shifter(degrees(shift_x_rad[i]), degrees(shift_y_rad[i])) for i in range(len(rotate_rad))]
    return [p for (p, ok) in result if ok]

def footprint_json(footprint):
    json = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {
                    "scalerank": 5,
                    "featurecla": "Footprint 1"
                },
                "geometry": {
                    "type": "LineString",
                    "coordinates": [
                        [
                            point.lon_d,
                            point.lat_d                        
                        ]
                        for point in footprint
                    ] +
                    [
                        [
                            footprint[0].lon_d,
                            footprint[0].lat_d      
                        ]
                    ]
                }
            }
        ]
    }
    
    return json

def add_marker(m, spaceobject):
    folium.Marker([spaceobject.lat_d, spaceobject.lon_d]).add_to(m)    
  
def render_footprint(m, sat, beam):       
    fp = footprint(sat, beam)    
    folium.GeoJson(footprint_json(fp), name='geojson').add_to(m)

def render_beam(m, lat_d = 0.0, lon_d = 0.0, beam = None):
    if beam == None:
        beam = Beam()
        beam.lat_d = lat_d
        beam.lon_d = lon_d

    add_marker(m, beam)
    render_footprint(m, sat, beam)    

In [3]:
m = folium.Map(location=[55.0, 37.0], zoom_start=5)    
    
sat = SpaceObject()
sat.lat_d = 0.0
sat.lon_d = 120.0
sat.alt = 35786*KM

beam_0 = Beam()
beam_0.lat_d = sat.lat_d
beam_0.lon_d = sat.lon_d

render_beam(m, beam=beam_0)

[
    render_beam(m, lat_d=65.0, lon_d=-70.0 + delta*11)
    for delta in range(9)
]

[
    render_beam(m, lat_d=55.0, lon_d=-70.0 + delta*11)
    for delta in range(9)
]

folium.LayerControl().add_to(m)

In [4]:
# m
m